In [ ]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from nltk.sentiment import SentimentIntensityAnalyzer
import operator

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk_stopwords= set(stopwords.words('english'))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
df= pd.read_json('/content/drive/MyDrive/amazon_data_scrapping/Data files/product2_reviews_details.json', orient='index')
df.head()

,reviewDate,reviewRating,reviewText,reviewTitle
0,"Reviewed in the United States on February 4, 2020",2.0 out of 5 stars,"This is a beautiful block that allows you to show your nice knives in a cool way. It worked great for my larger knives, if anything the magnets were almost overly aggressive on them. But for some reason it really struggled on my 6” chef’s knives or smaller. I have a set of Japanese VG10 kitchen knives and an alternate set of IKEA blades that I go back forth between depending on what I’m cooking or doing. The big chefs knives and santoku’s clung well for both sets. When we got to the smaller knives in either set they either hung loosely and actually twisted clanging into the others or didn’t stick at all in the case of either paring knife. Not sure if there aren’t enough magnets and they weren’t positioned well enough to catch them or what. Pretty holder, and was great for the large blades, but fell short for displaying full sets.","Good for large knives, not so much for smaller"
1,"Reviewed in the United States on January 18, 2021",5.0 out of 5 stars,"I love the magnetic knife block for :\n-its slim silhouette ( it freed up some space on my counter )\n- for its looks and for its safety acrylic shield ( from my cats who surely get on the counter when I’m not around)\n\nI thought long and hard before I decided on this knife block and I love it.\nIn the pictures, I showed both sides of the block. I was able to put, not only my working knives but also my 6 steak knives! It might be overcrowded for some, but I love having all these knives handy and it freed up a drawer!",This is the best knife block. Look no further!
2,"Reviewed in the United States on September 23, 2020",5.0 out of 5 stars,"I was getting really tired of two blocky, dirt-collecting wooden knife blocks reminding me of the 1980s on the kitchen island. As I am doing more at-home cooking now, I also wanted to reduce clutter. And as a fan of cooking shows, I respect the advice of certain chefs both professional and home. So, I looked into reducing the number of knives and reducing their footprint on my valuable but limited counter space while also adding a bit of style.\n\nI still have most of the old knives and steak knives, in a drawer for those few times when I might need them. Here's the trick for me: the Secura can only hold a few items and that is training me to use fewer tools to accomplish cooking tasks. This moves me to take more care and consideration in how I prepare meals and overall, I am more efficient.\n\nIt's only a knife holder, but it's a vital part of how I do at-home chef duties.",More than a countertop storage solution
3,"Reviewed in the United States on October 7, 2020",5.0 out of 5 stars,"My husband bought me a nice set of kitchen knives and ordered the magnetic knife holder to put them on. Works beautifully, and I'm happy with how the knives are presented on the knife holder. Very high quality magnetic wood and the glass shield is a nice accent for more protection to the knives.",A nice edition to the kitchen accessories!
4,"Reviewed in the United States on October 9, 2020",5.0 out of 5 stars,"I just replaced my plastic rod insert knife holder with this amazing magnetic one. With the old one, all of my knives couldn’t fit. They would poke out of the top. I cut myself once reaching for a knife because of that. Also, the plastic insert made me have to constantly pull out a knife to see if I got the right one since the knife blade was in the block unseen. Plus, there was no way to pull out the insert and clean it.\n\nThis magnetic block exceeds expectations! It takes up less counter space than my old block. It is slim and sleek. It looks like more expensive blocks that sell for hundreds of dollars. I love the see through ability. No more guessing which knife I’m pulling out. It comes with brushes to clean it. If you have mismatched knives like I do, this block handles them. I was able to fit 14 knives!",Slim and sleek


In [ ]:
df['reviewDate']= df['reviewDate'].apply(lambda date:re.sub('Reviewed in the United States on ',"", date).strip())
df['reviewRating']= df['reviewRating'].apply(lambda rating: re.sub(' out of 5 stars',"", rating).strip()) 
df['reviewRating']= df.reviewRating.astype('float')
df['reviewClass']=df.apply(lambda row: 1 if row.reviewRating>=4.0 else 0, axis=1)
# df['reviewText']= df['reviewTitle'] + '. '+ df['reviewText']
df['reviewText']= df['reviewText'].apply(lambda review: review.lower())
df['reviewText'] = df['reviewText'].apply(lambda review:re.sub("\n", ' ', review))

#shortforms
df['reviewText']= df['reviewText'].apply(lambda review: re.sub('bc|b/c', 'because', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("wasn’t", 'was not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("i’m", 'i am', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("i'm", 'i am', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("doesn’t", 'does not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("doesn't", 'does not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("didn’t", 'did not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("won't", 'will not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("won’t", 'will not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("don’t", 'do not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("can't", 'can not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("it's", 'it is', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("it’s", 'it is', review))
# df['reviewText']= df['reviewText'].apply(lambda review: re.sub("(\\d|\\W)+", ' ', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("(\\d)+", ' ', review))

In [ ]:
df.head()

,reviewDate,reviewRating,reviewText,reviewTitle
0,"February 4, 2020",2.0,"good for large knives, not so much for smaller. this is a beautiful block that allows you to show your nice knives in a cool way. it worked great for my larger knives, if anything the magnets were almost overly aggressive on them. but for some reason it really struggled on my ” chef’s knives or smaller. i have a set of japanese vg kitchen knives and an alternate set of ikea blades that i go back forth between depending on what i am cooking or doing. the big chefs knives and santoku’s clung well for both sets. when we got to the smaller knives in either set they either hung loosely and actually twisted clanging into the others or did not stick at all in the case of either paring knife. not sure if there aren’t enough magnets and they weren’t positioned well enough to catch them or what. pretty holder, and was great for the large blades, but fell short for displaying full sets.","Good for large knives, not so much for smaller"
1,"January 18, 2021",5.0,"this is the best knife block. look no further!. i love the magnetic knife block for : -its slim silhouette ( it freed up some space on my counter ) - for its looks and for its safety acrylic shield ( from my cats who surely get on the counter when i am not around) i thought long and hard before i decided on this knife block and i love it. in the pictures, i showed both sides of the block. i was able to put, not only my working knives but also my steak knives! it might be overcrowded for some, but i love having all these knives handy and it freed up a drawer!",This is the best knife block. Look no further!
2,"September 23, 2020",5.0,"more than a countertop storage solution. i was getting really tired of two blocky, dirt-collecting wooden knife blocks reminding me of the s on the kitchen island. as i am doing more at-home cooking now, i also wanted to reduce clutter. and as a fan of cooking shows, i respect the advice of certain chefs both professional and home. so, i looked into reducing the number of knives and reducing their footprint on my valuable but limited counter space while also adding a bit of style. i still have most of the old knives and steak knives, in a drawer for those few times when i might need them. here's the trick for me: the secura can only hold a few items and that is training me to use fewer tools to accomplish cooking tasks. this moves me to take more care and consideration in how i prepare meals and overall, i am more efficient. it is only a knife holder, but it is a vital part of how i do at-home chef duties.",More than a countertop storage solution
3,"October 7, 2020",5.0,"a nice edition to the kitchen accessories!. my husband bought me a nice set of kitchen knives and ordered the magnetic knife holder to put them on. works beautifully, and i am happy with how the knives are presented on the knife holder. very high quality magnetic wood and the glass shield is a nice accent for more protection to the knives.",A nice edition to the kitchen accessories!
4,"October 9, 2020",5.0,"slim and sleek. i just replaced my plastic rod insert knife holder with this amazing magnetic one. with the old one, all of my knives couldn’t fit. they would poke out of the top. i cut myself once reaching for a knife because of that. also, the plastic insert made me have to constantly pull out a knife to see if i got the right one since the knife blade was in the block unseen. plus, there was no way to pull out the insert and clean it. this magnetic block exceeds expectations! it takes up less counter space than my old block. it is slim and sleek. it looks like more expensive blocks that sell for hundreds of dollars. i love the see through ability. no more guessing which knife i am pulling out. it comes with brushes to clean it. if you have mismatched knives like i do, this block handles them. i was able to fit knives!",Slim and sleek


In [ ]:
def get_sentiment(score):
  if score>= 0.05:
    return 1
  else:
    return 0

sia_object= SentimentIntensityAnalyzer()
df['compound']= df['reviewTitle'].apply(lambda review: sia_object.polarity_scores(review)['compound'])
df['sentiment']= df['compound'].apply(lambda score: get_sentiment(score))

In [ ]:
df.head()

,reviewDate,reviewRating,reviewText,reviewTitle,compound,sentiment
0,"February 4, 2020",2.0,"good for large knives, not so much for smaller. this is a beautiful block that allows you to show your nice knives in a cool way. it worked great for my larger knives, if anything the magnets were almost overly aggressive on them. but for some reason it really struggled on my ” chef’s knives or smaller. i have a set of japanese vg kitchen knives and an alternate set of ikea blades that i go back forth between depending on what i am cooking or doing. the big chefs knives and santoku’s clung well for both sets. when we got to the smaller knives in either set they either hung loosely and actually twisted clanging into the others or did not stick at all in the case of either paring knife. not sure if there aren’t enough magnets and they weren’t positioned well enough to catch them or what. pretty holder, and was great for the large blades, but fell short for displaying full sets.","Good for large knives, not so much for smaller",0.4404,1
1,"January 18, 2021",5.0,"this is the best knife block. look no further!. i love the magnetic knife block for : -its slim silhouette ( it freed up some space on my counter ) - for its looks and for its safety acrylic shield ( from my cats who surely get on the counter when i am not around) i thought long and hard before i decided on this knife block and i love it. in the pictures, i showed both sides of the block. i was able to put, not only my working knives but also my steak knives! it might be overcrowded for some, but i love having all these knives handy and it freed up a drawer!",This is the best knife block. Look no further!,0.1007,1
2,"September 23, 2020",5.0,"more than a countertop storage solution. i was getting really tired of two blocky, dirt-collecting wooden knife blocks reminding me of the s on the kitchen island. as i am doing more at-home cooking now, i also wanted to reduce clutter. and as a fan of cooking shows, i respect the advice of certain chefs both professional and home. so, i looked into reducing the number of knives and reducing their footprint on my valuable but limited counter space while also adding a bit of style. i still have most of the old knives and steak knives, in a drawer for those few times when i might need them. here's the trick for me: the secura can only hold a few items and that is training me to use fewer tools to accomplish cooking tasks. this moves me to take more care and consideration in how i prepare meals and overall, i am more efficient. it is only a knife holder, but it is a vital part of how i do at-home chef duties.",More than a countertop storage solution,0.3182,1
3,"October 7, 2020",5.0,"a nice edition to the kitchen accessories!. my husband bought me a nice set of kitchen knives and ordered the magnetic knife holder to put them on. works beautifully, and i am happy with how the knives are presented on the knife holder. very high quality magnetic wood and the glass shield is a nice accent for more protection to the knives.",A nice edition to the kitchen accessories!,0.4753,1
4,"October 9, 2020",5.0,"slim and sleek. i just replaced my plastic rod insert knife holder with this amazing magnetic one. with the old one, all of my knives couldn’t fit. they would poke out of the top. i cut myself once reaching for a knife because of that. also, the plastic insert made me have to constantly pull out a knife to see if i got the right one since the knife blade was in the block unseen. plus, there was no way to pull out the insert and clean it. this magnetic block exceeds expectations! it takes up less counter space than my old block. it is slim and sleek. it looks like more expensive blocks that sell for hundreds of dollars. i love the see through ability. no more guessing which knife i am pulling out. it comes with brushes to clean it. if you have mismatched knives like i do, this block handles them. i was able to fit knives!",Slim and sleek,0.0000,0


In [ ]:
#120 positive reviews
df[df['sentiment']==1].shape

(120, 6)

In [ ]:
#40 negative reviews
df[df['sentiment']==0].shape

(40, 6)

In [ ]:
reviews=df[df['sentiment']==1]['reviewText']

positive_reviews=''
for review in reviews:
  positive_reviews= positive_reviews+" "+review

In [ ]:
reviews=df[df['sentiment']==0]['reviewText']

negative_reviews=''
for review in reviews:
  negative_reviews= negative_reviews+" "+review

In [ ]:
def create_frequency_matrix(reviews):
  frequency_matrix={}
  sentences= sent_tokenize(reviews)
  for sentence in sentences:
    words= word_tokenize(sentence)
    porter_stemmer= PorterStemmer()
    lemmatizer= WordNetLemmatizer()
    frequency_table= {}
    for word in words:
      word= porter_stemmer.stem(word)
      if word in nltk_stopwords:
        continue
      if word in frequency_table:
        frequency_table[word]+= 1
      else:
        frequency_table[word]=1
    frequency_matrix[sentence[:15]] = frequency_table
  return frequency_matrix


In [ ]:
def create_tf_matrix(frequency_matrix):
  tf_matrix={}
  for sentence, frequency_table in frequency_matrix.items():
    tf_table={}
    word_count_in_sentence= len(frequency_table)
    for word, frequency in frequency_table.items():
      tf_table[word]= frequency/word_count_in_sentence
    tf_matrix[sentence] = tf_table
  return tf_matrix

In [ ]:
def create_documents_per_words(frequency_matrix):
    word_per_doc_table = {}

    for sentence, frequency_table in frequency_matrix.items():
        for word, count in frequency_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [ ]:
import math
def create_idf_matrix(frequency_matrix, count_documents_per_words,total_documents):
    idf_matrix = {}

    for sentence, frequency_table in frequency_matrix.items():
        idf_table = {}

        for word in frequency_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_documents_per_words[word]))

        idf_matrix[sentence] = idf_table

    return idf_matrix

In [ ]:
def create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [ ]:
def rank_sentences(tf_idf_matrix):
  sentence_scores= {}
  for sentence, frequency_table in tf_idf_matrix.items():
   total_score=0
   words_in_sentence= len(frequency_table)
   for word, score in frequency_table.items():
    total_score += score

    sentence_scores[sentence] = total_score / words_in_sentence
  return sentence_scores

In [ ]:
def generate_summary(sentence_scores, positive_reviews):
  total_sum=0
  summary= ''
  
  for value in sentence_scores.values():
    total_sum=total_sum+value
  cut_off_score= total_sum// len(sentence_scores)

  cut_off_score= 1.5 * cut_off_score

  sentences= sent_tokenize(positive_reviews)
  for sentence in sentences:
    if sentence[:20] in sentence_scores and sentence_scores[sentence[:20]] >cut_off_score:
      summary+= ' '+ sentence
  
  return summary

In [ ]:
frequency_matrix= create_frequency_matrix(positive_reviews)
tf_matrix=create_tf_matrix(frequency_matrix)
count_documents_per_words=create_documents_per_words(frequency_matrix)
idf_matrix=create_idf_matrix(frequency_matrix, count_documents_per_words, len(sent_tokenize(positive_reviews)))
tf_idf_matrix=create_tf_idf_matrix(tf_matrix, idf_matrix)
sentence_scores=rank_sentences(tf_idf_matrix)
tf_idf_summary=generate_summary(sentence_scores, positive_reviews)

In [ ]:
tf_idf_summary

" buy it! impressive!!! love love love. heavy. it was perfect! excellent!. amazing buy!. it is amazing. update:  . . i love it. awesome!. great! buy it!. elegant. lovely! perfect. great product! it is worth it. love it!. love this! wonderful!. good quality. best i've seen. excellent!. stars from me! love it. love it!. good size. saves space. looks cool. great product. well made. sturdy. very well made. great product. good product. useful product. . . love this!. a++++. will buy again. love it!. great product. great product. good. recommended! we love it! love love love. looks great. a great buy!. great product. clean design! love!!. stunning. sturdy. very magnetic"

In [ ]:
len(tf_idf_summary)

671

In [ ]:
#change to camelcase
#apply spell check
#clean up extra spaces and periods
